In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
import phoenix as px

px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [10]:
from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor


endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))
trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

In [11]:
%cd /teamspace/studios/this_studio/final_project/Trading_Project

/teamspace/studios/this_studio/final_project/Trading_Project


In [12]:

from utils.file_text_handler import load_file, get_code_from_text
from utils.my_error_messages import extract_error_message
import dspy
import dsp
import cohere
import json
import functools
from dotenv import load_dotenv
import os
import requests
from datetime import datetime
import backtrader as bt
from backtrader import Indicator
# load_dotenv('.env')

text = load_file("cleaned_text.txt")

texts = text.split("\n\n\n\n")

In [13]:
base_strategy_PATH ="base_strategy_improved.py"
backtrader_examples_PATH = "backtrader_examples.py"
custom_examples_PATH = "custom_examples.py"
base_strats = load_file(base_strategy_PATH)
backtrader_examples = load_file(backtrader_examples_PATH)
custom_examples =load_file(custom_examples_PATH)
list_indicators = load_file("indicators.txt")



instruction = f"""
You are a python developer that intent to make a workable trading strategy. Your tasks are :
- Create a `CustomIndicator` class that inherit from the `Indicator` class
- Create a `BackTestStrategy` class that inherit from the `BaseStrategy` class and modify the `execute` function to follow human requirements.
Note : You MUST STRICTLY follow the instructions above.
Here is the `BaseStrategy` class : 
```python\n{base_strats}```

Here is the examples using price volume trend indicator :
```python\n{custom_examples}```

"""


In [14]:
from dotenv import load_dotenv
load_dotenv("/teamspace/studios/this_studio/sentiment_analysis/.env")

True

In [15]:
# Get example
import pandas as pd
import re
from my_dspy.dspy_signature import FinanceStrategyGenerator
from my_dspy.dspy_data import CSVDataset

FinanceStrategyGenerator.__doc__= instruction
    
# file_path = "Data/complex_trading_strategies.csv"
file_path = "Data/querstion_llm.csv"
dataset = CSVDataset(file_path=file_path, instruction=instruction)

In [16]:
# better net liquidation value view
from utils.errors_handler import error_tracking_decorator

class MyBuySell(bt.observers.BuySell):
    plotlines = dict(
        buy=dict(marker='$\u21E7$', markersize=12.0),
        sell=dict(marker='$\u21E9$', markersize=12.0)
    )

class CelebroCreator:
    def __init__(self, strategy, list_of_data, stake=100, cash=20000):
        # Initial cerebro
        self.cerebro = bt.Cerebro(cheat_on_open=True)
        
        self.cerebro.addstrategy(strategy)
        self.cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
        self.cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')
        self.cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns')
        self.cerebro.addobserver(bt.observers.Value)

        for data in list_of_data:
            self.cerebro.adddata(data)
        self.cerebro.broker.set_cash(cash)
        bt.observers.BuySell = MyBuySell
        
        # CURRENT
        self.strats = None
        _, self.message = self._run_cerebro()

    
    # CURRENT
    @error_tracking_decorator
    def _run_cerebro(self):
        self.strats = self.cerebro.run()


    def show(self):
        print("Final Portfolio Value: %.2f" % self.cerebro.broker.getvalue())
        print("Total point return: ", (self.cerebro.broker.getvalue() - self.cerebro.broker.startingcash))

        try:
            sharpe_ratio = self.strats[0].analyzers.SharpeRatio.get_analysis()['sharperatio']
            print('Sharpe Ratio:', sharpe_ratio)
        except:
            print('No Buy/Sell Signal! No Sharpe Ratio!')
        # Plot the results
        figs = self.cerebro.plot(
            iplot=False, 
            # style="pincandle", 
            # width=60 * 10, height=40 * 10,
            figsize=(100, 80),
            # sharpe_ratio=sharpe_ratio
        )
        return figs

In [17]:
import traceback
import sys
from prompt_retry import prompt_error_template1, prompt_error_template2
from utils.file_text_handler import save_file
from utils.errors_handler import get_error


def check_valid_code(strategy, list_data):
    
    obj = CelebroCreator(strategy,list_data)
    count = {}
    
    if obj.strats is not None:
      count['BuySignal'] = obj.strats[0].cbuy
      count['SellSignal'] = obj.strats[0].csell
      
    message = obj.message
    errors = get_error(message) if message else ["",""]
    
    return errors, count



def check_valid_indicators(**kwargs):
    if kwargs['countBuy'] > 1 and \
       kwargs['countSell'] > 1:
        return True
    return False
    


class GenerateCodeWithAssert(dspy.Module):
  def __init__(self, list_ohcl_data):
    super().__init__()
    self.generate_result = dspy.ChainOfThought(FinanceStrategyGenerator)
    self.ohcl_data = list_ohcl_data
    self.num_retry = 0
    self.flag = 0
    self.complete = False
    self.still_errors = False
    self.max_retry = 8

  
  def forward(self, question):
    
    # Get answer
    ex = self.generate_result(question=question)
    
      
    if self.flag == 0:
        self.flag = 1
    else:
        self.num_retry += 1
        
    # Get and execute code
    exec(get_code_from_text(ex.answer), globals())
    

    # Extract Error
    errors,countSignal = check_valid_code(BackTestStrategy, self.ohcl_data)
    # -------------------
    
    # Check error in the code answer
    check = True if errors[0] == "" else False
    
    if not check:
        p_error = prompt_error_template1(errors=errors) if errors[-1] == "" else prompt_error_template2(errors=errors)
    else:
        p_error = ""
    
    dspy.Suggest(check, f"{p_error}")

    
    # Check the number of buy and sell signals
    check1 = False
    if countSignal:
      check1 = check_valid_indicators(countBuy=countSignal['BuySignal'], countSell=countSignal['SellSignal'])
      
      dspy.Suggest(check1, f"Please review and correct the formulas and conditions. Make sure the strategy includes at least one buy and one sell signal.")
    # ---------
    
    
    ex['num_retry'] = self.num_retry
    

    self.complete = True ex['num_retry'] <= self.max_retry and check1 == True else False
    self.still_errors = True if ex['num_retry'] == self.max_retry and check == False else False
    
    
    
    ex['status'] = {
      "Complete": self.complete,
      "Still_Error": self.still_errors
    }
    
    # Reset attributes
    self.num_retry, self.flag = 0, 0
    self.still_errors, self.complete = False, False
    
    return ex


In [80]:
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
from data_loader import load_stock_data
from base_strategy_improved import BaseStrategy
from dspy.predict import Retry
import backtrader as bt
import dsp
import random
random.seed(42)


# Anyscale
lm = dspy.Anyscale(
    model="meta-llama/Meta-Llama-3-70B-Instruct",
    max_tokens=2048, 
    use_chat_api=True,
    temperature=0.0
)


dspy.settings.configure(lm=lm, trace=[])

data = [bt.feeds.PandasData(
                dataname=load_stock_data(ticker='AAPL', period="1y"), datetime="Date", 
                timeframe=bt.TimeFrame.Minutes)]

generate_with_assert = assert_transform_module(GenerateCodeWithAssert(list_ohcl_data=data).map_named_predictors(Retry), functools.partial(backtrack_handler, max_backtracks=8))


query = "Formulate a strategy to buy when the Gann Angles indicate support at a key level and the 14-day RSI is above 50 during a bullish market. Define sell conditions for when the Gann Angles indicate resistance at a key level and the RSI falls below 50."
# example = generate_with_assert(dataset.train[0].question)
example = generate_with_assert(query)

# print(f"Question: {dataset.train[0].question}")
print(f"Question: {query}")
print(f"Final Predicted Answer (after CoT process): {example.answer}")
print(f"Number of Retries: {example.num_retry}")

[*********************100%%**********************]  1 of 1 completed


ERROR [dspy.primitives.assertions] 2024-07-04T16:25:30.839883Z [error    ] SuggestionFailed: 
        The code must avoid the AttributeError: 'LineBuffer' object has no attribute 'diff'. This error occurs in the following function:
 Function name: next, params: {}
Error location: Line: 25,  [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR [dspy.primitives.assertions] 2024-07-04T16:25:45.424765Z [error    ] SuggestionFailed: 
    You encountered an error from the package `ERROR MESSEAGE`: `IndexError: array index out of range` at `Function name: __getitem__, params: {'ago': 0}`. 
    
    This error originates from the code in your function 
 Function name: next, params: {}
The error occurs at `Line: 23,` of your code answer. The code should not contain this error. 
    Please retry with the following description."
     [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR [dspy.primitives.assertions] 2024-07-04T16:26:02.413669Z [error    ] Suggestion

2023-07-28T00:05:00, SELL EXECUTED, Price: 194.67, Cost: -194.67, Comm 0.00


ERROR [dspy.primitives.assertions] 2024-07-04T16:26:33.489630Z [error    ] SuggestionFailed: Please review and correct the formulas and conditions. Make sure the strategy includes at least one buy and one sell signal. [dspy.primitives.assertions] filename=assertions.py lineno=111


2023-07-28T00:05:00, SELL EXECUTED, Price: 194.67, Cost: -194.67, Comm 0.00


...

2023-07-25T00:05:00, SELL EXECUTED, Price: 193.33, Cost: -193.33, Comm 0.00


ERROR [dspy.primitives.assertions] 2024-07-04T16:27:24.739153Z [error    ] SuggestionFailed: Please review and correct the formulas and conditions. Make sure the strategy includes at least one buy and one sell signal. [dspy.primitives.assertions] filename=assertions.py lineno=111


2023-07-25T00:05:00, SELL EXECUTED, Price: 193.33, Cost: -193.33, Comm 0.00


ERROR [dspy.primitives.assertions] 2024-07-04T16:27:41.256008Z [error    ] SuggestionFailed: Please review and correct the formulas and conditions. Make sure the strategy includes at least one buy and one sell signal. [dspy.primitives.assertions] filename=assertions.py lineno=108


2023-07-25T00:05:00, SELL EXECUTED, Price: 193.33, Cost: -193.33, Comm 0.00
Question: Formulate a strategy to buy when the Gann Angles indicate support at a key level and the 14-day RSI is above 50 during a bullish market. Define sell conditions for when the Gann Angles indicate resistance at a key level and the RSI falls below 50.
Final Predicted Answer (after CoT process): ```python
import backtrader as bt

class GannAngle(bt.Indicator):
    lines = ("angle",)
    params = (("period", 1),)

    def __init__(self):
        self.addminperiod(self.params.period)

    def next(self):
        if len(self) > 1:
            self.lines.angle[0] = self.data.close[-1] * (1 + 0.618 ** (self.p.period - 1))
        else:
            self.lines.angle[0] = self.data.close[0]

class RSI(bt.Indicator):
    lines = ("rsi",)
    params = (("period", 14),)

    def __init__(self):
        self.addminperiod(self.params.period)
        self.up, self.down = 0, 0
        self.roll_up, self.roll_down = 0, 0


In [59]:
lm.inspect_history(n=3)




You are a python developer that intent to make a workable trading strategy. Your tasks are :
- Create a `CustomIndicator` class that inherit from the `Indicator` class
- Create a `BackTestStrategy` class that inherit from the `BaseStrategy` class and modify the `execute` function to follow human requirements.
Note : You MUST STRICTLY follow the instructions above.
Here is the `BaseStrategy` class : 
```python
import backtrader as bt


# Define a base strategy class for handling buy and sell signals and order notifications.
class BaseStrategy(bt.Strategy):
    def __init__(self, debug=True):
        """
        Initialize the strategy with parameters.

        Args:
        - debug (bool): If True, enable debug logging.

        Attributes:
        - countBuy (int): Counter for buy signals.
        - countSell (int): Counter for sell signals.
        - final_signal (int or None): Final signal for trading: 1 (long), 0 (neutral), -1 (sell).
        - debug (bool): Flag for debug mode.


'\n\n\nYou are a python developer that intent to make a workable trading strategy. Your tasks are :\n- Create a `CustomIndicator` class that inherit from the `Indicator` class\n- Create a `BackTestStrategy` class that inherit from the `BaseStrategy` class and modify the `execute` function to follow human requirements.\nNote : You MUST STRICTLY follow the instructions above.\nHere is the `BaseStrategy` class : \n```python\nimport backtrader as bt\n\n\n# Define a base strategy class for handling buy and sell signals and order notifications.\nclass BaseStrategy(bt.Strategy):\n    def __init__(self, debug=True):\n        """\n        Initialize the strategy with parameters.\n\n        Args:\n        - debug (bool): If True, enable debug logging.\n\n        Attributes:\n        - countBuy (int): Counter for buy signals.\n        - countSell (int): Counter for sell signals.\n        - final_signal (int or None): Final signal for trading: 1 (long), 0 (neutral), -1 (sell).\n        - debug (bo

In [ ]:
import matplotlib
%matplotlib inline

result = CelebroCreator(BackTestStrategy,data)
result.show()


# Bootstrap find the best few-shots 

In [83]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.evaluate import Evaluate
from my_dspy.dspy_metric import validate_answer



teleprompter = BootstrapFewShotWithRandomSearch(metric = validate_answer, max_bootstrapped_demos=3, num_candidate_programs=6)


generated_code_student_teacher = teleprompter.compile(
                student=assert_transform_module(
                    GenerateCodeWithAssert(list_ohcl_data=data).map_named_predictors(Retry), functools.partial(backtrack_handler, max_backtracks=8)), 
                teacher = assert_transform_module(
                    GenerateCodeWithAssert(list_ohcl_data=data).map_named_predictors(Retry), functools.partial(backtrack_handler, max_backtracks=5)), 
                trainset=dataset.train[:30], 
                valset=dataset.dev[:5]

)

  0%|          | 0/5 [00:00<?, ?it/s]

ERROR [dspy.primitives.assertions] 2024-07-04T16:49:30.818743Z [error    ] SuggestionFailed: 
    You encountered an error from the package `ERROR MESSEAGE`: `TypeError: MACDHisto.__init__() got an unexpected keyword argument 'fast'` at `Function name: doinit, params: {'cls': "<class 'backtrader.indicators.ma...", '_obj': '<backtrader.indicators.macd.MACD...', 'args': (), 'kwargs': "{'fast': 12, 'slow': 26, 'signal..."}`. 
    
    This error originates from the code in your function 
 Function name: __init__, params: {}
The error occurs at `Line: 10,` of your code answer. The code should not contain this error. 
    Please retry with the following description."
     [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR [dspy.primitives.assertions] 2024-07-04T16:49:40.972815Z [error    ] SuggestionFailed: 
    You encountered an error from the package `ERROR MESSEAGE`: `TypeError: MACDHisto.__init__() got an unexpected keyword argument 'fastperiod'` at `Function name: do

Example Prediction(
    answer='```python\nimport backtrader as bt\n\nclass CustomIndicator(bt.Indicator):\n    lines = ("rsi", "macd", "signal", "macd_hist")\n    params = (("rsi_period", 14), ("macd_fast", 12), ("macd_slow", 26), ("macd_signal", 9))\n\n    def __init__(self):\n        self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.params.rsi_period)\n        self.macd, self.signal, self.macd_hist = bt.indicators.MACD(self.data.close, \n                                                                   fast=self.params.macd_fast, \n                                                                   slow=self.params.macd_slow, \n                                                                   signal=self.params.macd_signal)\n    def next(self):\n        self.lines.rsi[0] = self.rsi[0]\n        self.lines.macd[0] = self.macd[0]\n        self.lines.signal[0] = self.signal[0]\n        self.lines.macd_hist[0] = self.macd_hist[0]\n\nclass BackTestStrategy(BaseStrategy):\n   

Average Metric: 1 / 2  (50.0):  40%|████      | 2/5 [01:42<02:12, 44.21s/it]

2023-08-07T00:05:00, SELL EXECUTED, Price: 182.13, Cost: -182.13, Comm 0.00
2023-09-01T00:05:00, CLOSE SELL, Price: 189.49, Cost: -182.13, Comm 0.00
2023-09-05T00:05:00, BUY EXECUTED, Price: 188.28, Cost: 188.28, Comm 0.00
2023-10-27T00:05:00, CLOSE BUY, Price: 166.91, Cost: 188.28, Comm 0.00
2023-11-13T00:05:00, BUY EXECUTED, Price: 185.82, Cost: 185.82, Comm 0.00
2024-01-03T00:05:00, CLOSE BUY, Price: 184.22, Cost: 185.82, Comm 0.00
2024-01-04T00:05:00, SELL EXECUTED, Price: 182.15, Cost: -182.15, Comm 0.00
2024-05-06T00:05:00, CLOSE SELL, Price: 182.35, Cost: -182.15, Comm 0.00
2024-06-12T00:05:00, BUY EXECUTED, Price: 207.37, Cost: 207.37, Comm 0.00
Example Prediction(
    rationale='design a trading strategy that incorporates Bollinger Bands and On Balance Volume (OBV) indicators. We need to create a custom indicator for OBV and then define the trading logic in the `execute` method of the `BackTestStrategy` class.',
    answer='```python\nimport backtrader as bt\n\n\nclass OBV(bt.

...

2023-08-29T00:05:00, SELL EXECUTED, Price: 179.70, Cost: -179.70, Comm 0.00
2023-12-27T00:05:00, CLOSE SELL, Price: 192.49, Cost: -179.70, Comm 0.00
2024-01-02T00:05:00, BUY EXECUTED, Price: 187.15, Cost: 187.15, Comm 0.00
2024-03-21T00:05:00, CLOSE BUY, Price: 177.05, Cost: 187.15, Comm 0.00
2024-04-12T00:05:00, SELL EXECUTED, Price: 174.26, Cost: -174.26, Comm 0.00
Example Prediction(
    answer='```python\nimport backtrader as bt\n\nclass CustomIndicatorSAR(bt.Indicator):\n    lines = ("sar",)\n    params = (("acceleration", 0.02), ("maximum", 0.2))\n\n    def __init__(self):\n        self.addminperiod(1)\n\n    def next(self):\n        if len(self) == 1:\n            self.lines.sar[0] = self.data.close[0]\n        else:\n            if self.data.close[0] > self.lines.sar[-1]:\n                self.lines.sar[0] = self.lines.sar[-1] + self.params.acceleration * (self.data.close[0] - self.lines.sar[-1])\n            else:\n                self.lines.sar[0] = self.lines.sar[-1] - self.

ERROR [dspy.primitives.assertions] 2024-07-04T18:03:27.512570Z [error    ] SuggestionFailed: 
    You encountered an error from the package `ERROR MESSEAGE`: `TypeError: must be real number, not LinesOperation` at `Function name: __setitem__, params: {'ago': 0, 'value': '<backtrader.linebuffer.LinesOper...'}`. 
    
    This error originates from the code in your function 
 Function name: next, params: {'tp': 193.30333455403647, 'sma': '<backtrader.indicators.sma.SMA o...', 'md': '<backtrader.indicators.deviation...'}
The error occurs at `Line: 14,` of your code answer. The code should not contain this error. 
    Please retry with the following description."
     [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR [dspy.primitives.assertions] 2024-07-04T18:03:39.638829Z [error    ] SuggestionFailed: 
    You encountered an error from the package `ERROR MESSEAGE`: `IndexError: array index out of range` at `Function name: __getitem__, params: {'ago': 0}`. 
    
    This

Example Prediction(
    answer='Here is the corrected code:\n```python\nimport backtrader as bt\n\nclass CCI(bt.Indicator):\n    lines = ("cci",)\n    params = (("period", 14),)\n\n    def __init__(self):\n        self.addminperiod(self.params.period)\n\n    def next(self):\n        tp = (self.data.high[0] + self.data.low[0] + self.data.close[0]) / 3\n        sma = bt.indicators.SMA_simple(tp, period=self.params.period)\n        md = bt.indicators.StdDev_simple(tp, period=self.params.period)\n        self.lines.cci[0] = (tp - sma) / (0.015 * md)\n\nclass EMA(bt.Indicator):\n    lines = ("ema",)\n    params = (("period", 50),)\n\n    def __init__(self):\n        self.addminperiod(self.params.period)\n\n    def next(self):\n        self.lines.ema[0] = bt.indicators.EMA_simple(self.data.close, period=self.params.period)\n\nclass BackTestStrategy(BaseStrategy):\n    def __init__(self, *args, **kwargs):\n        super().__init__(*args, **kwargs)\n        self.cci = CCI()\n        self.ema50

ERROR [dspy.primitives.assertions] 2024-07-04T18:05:25.800574Z [error    ] SuggestionFailed: 
    You encountered an error from the package `ERROR MESSEAGE`: `IndexError: array index out of range` at `Function name: __getitem__, params: {'ago': 0}`. 
    
    This error originates from the code in your function 
 Function name: next, params: {}
The error occurs at `Line: 12,` of your code answer. The code should not contain this error. 
    Please retry with the following description."
     [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR [dspy.primitives.assertions] 2024-07-04T18:05:43.098134Z [error    ] SuggestionFailed: 
    You encountered an error from the package `ERROR MESSEAGE`: `IndexError: array index out of range` at `Function name: __getitem__, params: {'ago': 0}`. 
    
    This error originates from the code in your function 
 Function name: next, params: {}
The error occurs at `Line: 12,` of your code answer. The code should not contain this error. 


Example Prediction(
    answer='Here is the corrected code:\n\n```python\nimport backtrader as bt\n\nclass ADX(bt.Indicator):\n    lines = ("adx",)\n    params = (("period", 14),)\n\n    def __init__(self):\n        self.addminperiod(self.params.period)\n\n    def next(self):\n        if len(self) >= self.params.period:\n            up_move = self.data.high - self.data.high(-1)\n            down_move = self.data.low(-1) - self.data.low\n            up_move[up_move < 0] = 0\n            down_move[down_move < 0] = 0\n            tr = bt.indicators.Max((self.data.high - self.data.low), (self.data.high - self.data.close(-1)), (self.data.close(-1) - self.data.low))\n            tr_sum = bt.indicators.SumN(tr, period=self.params.period)\n            up_sum = bt.indicators.SumN(up_move, period=self.params.period)\n            down_sum = bt.indicators.SumN(down_move, period=self.params.period)\n            plus_di = 100 * (up_sum / tr_sum)\n            minus_di = 100 * (down_sum / tr_sum)\n   

In [86]:
generated_code_student_teacher.save("/teamspace/studios/this_studio/final_project/Trading_Project/module/new_code_generation_fewshot.json")

In [85]:
evaluate = Evaluate(
    devset = dataset.dev[-10:],
    metric = validate_answer,
    num_threads=3,
    display_progress=True,
    display_table=10
)

evaluate(generated_code_student_teacher)

  0%|          | 0/10 [00:00<?, ?it/s]

ERROR [dspy.primitives.assertions] 2024-07-04T18:11:42.915141Z [error    ] SuggestionFailed: 
    You encountered an error from the package `ERROR MESSEAGE`: `TypeError: MACD.__init__() got an unexpected keyword argument 'period'` at `Function name: doinit, params: {'cls': "<class 'backtrader.indicators.ma...", '_obj': '<backtrader.indicators.macd.MACD...', 'args': (), 'kwargs': {'period': 9}}`. 
    
    This error originates from the code in your function 
 Function name: __init__, params: {}
The error occurs at `Line: 39,` of your code answer. The code should not contain this error. 
    Please retry with the following description."
     [dspy.primitives.assertions] filename=assertions.py lineno=111
ERROR [dspy.primitives.assertions] 2024-07-04T18:12:00.239552Z [error    ] SuggestionFailed: Please review and correct the formulas and conditions. Make sure the strategy includes at least one buy and one sell signal. [dspy.primitives.assertions] filename=assertions.py lineno=111


2023-08-10T00:05:00, BUY EXECUTED, Price: 179.48, Cost: 179.48, Comm 0.00


ERROR [dspy.primitives.assertions] 2024-07-04T18:12:16.411071Z [error    ] SuggestionFailed: Please review and correct the formulas and conditions. Make sure the strategy includes at least one buy and one sell signal. [dspy.primitives.assertions] filename=assertions.py lineno=111


2023-08-10T00:05:00, BUY EXECUTED, Price: 179.48, Cost: 179.48, Comm 0.00


...

Example Prediction(
    answer='```python\nclass ChaikinOscillator(bt.Indicator):\n    lines = ("osc",)\n    params = (("period", 3), ("fast", 3), ("slow", 10))\n\n    def __init__(self):\n        self.addminperiod(self.params.period)\n        self.ema_fast = bt.ind.EMA(self.data.close, period=self.params.fast)\n        self.ema_slow = bt.ind.EMA(self.data.close, period=self.params.slow)\n        self.ad = bt.ind.AccDist_OSC(self.data, period=self.params.period)\n\n    def next(self):\n        self.lines.osc[0] = (self.ema_fast - self.ema_slow) + self.ad\n\nclass BackTestStrategy(BaseStrategy):\n    def __init__(self, *args, **kwargs):\n        super().__init__(*args, **kwargs)\n        self.chaikin_osc = ChaikinOscillator()\n        self.ma_50 = bt.ind.SMA(self.data.close, period=50)\n        self.ma_200 = bt.ind.SMA(self.data.close, period=200)\n\n    def execute(self):\n        if len(self) > 1:\n            if self.chaikin_osc.osc[0] > 0 and self.ma_50[0] > self.ma_200[0]:\n       

Average Metric: 2 / 10  (20.0): 100%|██████████| 10/10 [17:59<00:00, 107.96s/it]

2023-07-26T00:05:00, BUY EXECUTED, Price: 193.67, Cost: 193.67, Comm 0.00
2023-08-03T00:05:00, CLOSE BUY, Price: 191.57, Cost: 193.67, Comm 0.00
2023-08-04T00:05:00, SELL EXECUTED, Price: 185.52, Cost: -185.52, Comm 0.00
2023-08-30T00:05:00, CLOSE SELL, Price: 184.94, Cost: -185.52, Comm 0.00
2023-08-31T00:05:00, BUY EXECUTED, Price: 187.84, Cost: 187.84, Comm 0.00
2023-09-07T00:05:00, CLOSE BUY, Price: 175.18, Cost: 187.84, Comm 0.00
2023-09-08T00:05:00, SELL EXECUTED, Price: 178.35, Cost: -178.35, Comm 0.00
2023-10-09T00:05:00, CLOSE SELL, Price: 176.81, Cost: -178.35, Comm 0.00
2023-10-10T00:05:00, BUY EXECUTED, Price: 178.10, Cost: 178.10, Comm 0.00
2023-10-18T00:05:00, CLOSE BUY, Price: 175.58, Cost: 178.10, Comm 0.00
2023-10-19T00:05:00, SELL EXECUTED, Price: 176.04, Cost: -176.04, Comm 0.00
2023-11-03T00:05:00, CLOSE SELL, Price: 174.24, Cost: -176.04, Comm 0.00
2023-11-06T00:05:00, BUY EXECUTED, Price: 176.38, Cost: 176.38, Comm 0.00
2024-01-03T00:05:00, CLOSE BUY, Price: 184.2

,question,answer,num_retry,validate_answer,rationale
0,Develop a strategy to buy when the Fibonacci Retracement level of 61.8% is tested and the MACD histogram is positive during a market correction. Set...,"```python import backtrader as bt import numpy as np class FibonacciRetracement(bt.Indicator): lines = (""fib_retracement"",) params = ((""period"", 1),) def __init__(self): self.addminperiod(self.params.period) def next(self): if len(self)...",8,False,nan
1,Create a trading strategy that initiates a buy order when the Keltner Channels show a price break above the upper channel and the 14-day RSI...,"```python import backtrader as bt import pandas as pd class KeltnerChannel(bt.Indicator): lines = (""upper"", ""middle"", ""lower"") params = ((""period"", 20), (""mult"", 2)) def __init__(self): self.addminperiod(self.params.period)...",8,False,nan
2,Formulate a strategy to buy when the Donchian Channels show a price breakout above the upper channel and the ATR indicates increasing volatility during a...,"```python import backtrader as bt import numpy as np class DonchianChannels(bt.Indicator): lines = (""upper"", ""middle"", ""lower"") params = ((""period"", 20),) def __init__(self): self.addminperiod(self.params.period) def next(self):...",8,False,nan
3,Design a trading strategy that triggers a buy signal when the Aroon Up indicator crosses above the Aroon Down indicator and the 20-day SMA crosses...,"```python import backtrader as bt import numpy as np class AroonUp(bt.Indicator): lines = (""aroonup"",) params = ((""period"", 14),) def __init__(self): self.addminperiod(self.params.period) def next(self): high =...",8,False,nan
4,Develop a strategy to buy when the Ulcer Index indicates a low level of market risk and the 50-day EMA crosses above the 200-day EMA...,"```python import backtrader as bt import math class UlcerIndex(bt.Indicator): lines = (""ulcer"",) params = ((""period"", 14),) def __init__(self): self.addminperiod(self.params.period) def next(self): if len(self) > 1:...",8,False,nan
5,Create a trading strategy that initiates a buy order when the Vortex Indicator shows a positive trend (VI+ above VI-) and the 14-day RSI is...,"```python import backtrader as bt import numpy as np class VortexIndicator(bt.Indicator): lines = (""vip"", ""vim"") params = ((""period"", 14),) def __init__(self): self.addminperiod(self.params.period) def next(self): if...",8,False,nan
6,Formulate a strategy to buy when the DMI shows +DI above -DI and the 20-day EMA crosses above the 50-day EMA during a bullish market....,"```python import backtrader as bt class DMI(bt.Indicator): lines = (""pdi"", ""mdi"", ""adx"") params = ((""period"", 14),) def __init__(self): self.addminperiod(self.params.period) def next(self): if len(self) > self.params.period:...",8,False,nan
7,Design a trading strategy that triggers a buy signal when the Price ROC is positive and the 14-day RSI is above 50 during a market...,"```python import backtrader as bt import numpy as np class ROC(bt.Indicator): lines = (""roc"",) params = ((""period"", 1),) def __init__(self): self.addminperiod(self.params.period) def next(self): if len(self)...",0,✔️ [True],"produce the answer. We need to create a custom indicator for the Price Rate of Change (ROC) and use the existing RSI indicator. Then, we'll..."
8,Develop a strategy to buy when the Chaikin Oscillator is positive and the 50-day MA is above the 200-day MA during a bullish market. Set...,"```python class ChaikinOscillator(bt.Indicator): lines = (""osc"",) params = ((""period"", 3), (""fast"", 3), (""slow"", 10)) def __init__(self): self.addminperiod(self.params.period) self.ema_fast = bt.ind.EMA(self.data.close, period=self.params.fast) self.ema_slow = bt.ind.EMA(self.data.close, period=self.params.slow)...",8,False,nan
9,Create a trading strategy that initiates a buy order when the TRIX Indicator is positive and the 14-day RSI is above 50 during a bullish...,"``` import backtrader as bt import numpy as np class TRIX(

20.0

In [22]:
question = "Develop a strategy that triggers a buy signal when the Parabolic SAR indicates an uptrend and the Chaikin Money Flow (CMF) is above zero, indicating buying pressure in a bullish market. Define sell conditions for when the Parabolic SAR indicates a downtrend and the CMF is below zero, indicating selling pressure."
example = generated_code_student_teacher(question=question)
example.answer

"```python\nimport backtrader as bt\n\nclass ParabolicSAR(bt.Indicator):\n    lines = ('sar',)\n    params = (('acceleration', 0.02), ('maximum', 0.2))\n\n    def __init__(self):\n        self.addminperiod(1)\n\n    def next(self):\n        # Implement Parabolic SAR logic here\n        pass\n\nclass ChaikinMoneyFlow(bt.Indicator):\n    lines = ('cmf',)\n    params = (('period', 21),)\n\n    def __init__(self):\n        self.addminperiod(self.params.period)\n\n    def next(self):\n        # Implement Chaikin Money Flow logic here\n        pass\n\nclass BackTestStrategy(BaseStrategy):\n    def __init__(self, *args, **kwargs):\n        super().__init__(*args, **kwargs)\n        self.sar = ParabolicSAR()\n        self.cmf = ChaikinMoneyFlow()\n\n    def execute(self):\n        if self.sar.sar[0] < self.data.close[0] and self.cmf.cmf[0] > 0:\n            return 1  # Long signal\n        elif self.sar.sar[0] > self.data.close[0] and self.cmf.cmf[0] < 0:\n            return -1  # Short signal